# Gan Baseline

#### NOTE: Need to activate genomelake environment before this code. Simply type 'genomelake' in terminal.

In [1]:
%env CUDA_VISIBLE_DEVICES=4,5
import os, sys
sys.path.append("..")
import random
# custom file path package
from data import Data_Directories
# custom utility package
from utils.compute_util import *
# package for genomic data
from pybedtools import Interval, BedTool
from genomelake.extractors import ArrayExtractor, BigwigExtractor
# package for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats.stats import pearsonr,spearmanr
import tensorflow as tf

env: CUDA_VISIBLE_DEVICES=4,5


Using TensorFlow backend.


In [2]:
window_size = 2001
sample_num = 10000

In [3]:
# TODO: No pretrained model yet
# model_path = "models/cnn_2000_new/best_model.h5"

In [4]:
# retrieve data
data = Data_Directories()
print data.intervals.keys()
print data.input_atac['day0'].keys()
print data.output_histone['day0'].keys()

['day6', 'day3', 'day0']
['100', '140']
['H3K27me3', 'H3K4me1', 'H3K27ac']


In [5]:
# get intervals for day0 data
day0_intervals = list(BedTool(data.intervals['day0']))
print '# of Intervals Extracted for day0: {}'.format(len(day0_intervals))

# of Intervals Extracted for day0: 267226


In [6]:
# create an ArrayExtractor for ATAC-seq for day0 with 140 base pairs
bw_140bp_day0 = ArrayExtractor(data.input_atac['day0']['140'])
print 'Finished extracting bigwig for day0, 140bp'

Finished extracting bigwig for day0, 140bp


In [7]:
# create a BigWigExtractor for histone makr 'H3K27ac' for day0
bw_histone_mark_day0 = BigwigExtractor(data.output_histone['day0']['H3K27ac'])
print 'Finished extracting bigwig for day0, 140bp'

Finished extracting bigwig for day0, 140bp


In [8]:
# normalize day0 intervals
normalized_day0_intervals = [normalize_interval(interval, window_size) for interval in day0_intervals if normalize_interval(interval, window_size)]
print 'Finished normalizing day0 intervals!'

Finished normalizing day0 intervals!


In [9]:
assert (len(day0_intervals)==len(normalized_day0_intervals))
print "Examples of original intervals"
print [(int(_interval.start)+int(_interval[-1]), [int(_interval.start), int(_interval.end)])
       for _interval in day0_intervals[:3]]
print "Examples of normalized intervals with window size of {}".format(window_size)
print [([int(_interval.start), int(_interval.end)])
       for _interval in  normalized_day0_intervals[:3]]

Examples of original intervals
[(123412027, [123411855, 123412989]), (123411941, [123411855, 123412989]), (131908564, [131908487, 131910071])]
Examples of normalized intervals with window size of 2001
[[123411027, 123413028], [123410941, 123412942], [131907564, 131909565]]


In [10]:
atac_seq_day0 = bw_140bp_day0(normalized_day0_intervals)
print atac_seq_day0.shape

(267226, 2001, 5)


In [11]:
#TODO: put this into utils if possible
def prune_invalid_intervals(intervals, bigwig_file):
    for _interval in intervals[:]:
        try:
            bigwig_file([_interval])
        except:
            intervals.remove(_interval)
            pass
        
print "Before pruning day0: {}".format(len(normalized_day0_intervals))
prune_invalid_intervals(normalized_day0_intervals, bw_140bp_day0)
print "After pruning day0: {}".format(len(normalized_day0_intervals))

Before pruning day0: 267226
After pruning day0: 267226


In [12]:
print "Dimension of ATAC-seq signal: {}".format(bw_140bp_day0(normalized_day0_intervals[:1]).shape)

Dimension of ATAC-seq signal: (1, 2001, 5)


In [13]:
print "Dimension of histone mark signal: {}".format(bw_histone_mark_day0(normalized_day0_intervals[:1]).shape)

Dimension of histone mark signal: (1, 2001)


In [14]:
# Replace nan values with zeros and convert it to p-values
histone_mark_day0 = np.nan_to_num(bw_histone_mark_day0(normalized_day0_intervals))
print histone_mark_day0.shape

(267226, 2001)


In [15]:
histone_mark_day0 = np.expand_dims(histone_mark_day0, axis=2)
print histone_mark_day0.shape

(267226, 2001, 1)


In [16]:
print "Example histone mark signal"
print "\tRaw value: {}".format(bw_histone_mark_day0(normalized_day0_intervals[:1])[0][:5].reshape(-1))

Example histone mark signal
	Raw value: [ 0.01014  0.01014  0.01014  0.02435  0.02435]


In [17]:
from keras.layers import AveragePooling1D, Input, Dense, Conv1D, Dropout, BatchNormalization, Activation, ZeroPadding1D, Reshape, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras import optimizers
from keras import metrics
from keras import losses
from keras import backend as K
from keras.callbacks import Callback, TensorBoard, ReduceLROnPlateau, ModelCheckpoint
from keras.optimizers import Adam, SGD

In [18]:
smooth_rate = 0.1
dropout_rate = 0.5
# parameters for first conv layer
hidden_filters_1 = 32
hidden_kernel_size_1 = window_size
# parameters for second conv layer
output_filters = 1
output_kernel_size = 32
# parameters for training
batch_size = 128
num_epochs = 200
evaluation_freq = 10

In [19]:
# Helper functions for writing the scores into bigwig file
from itertools import izip
from itertools import groupby
import subprocess

def interval_key(interval):
    return (interval.chrom, interval.start, interval.stop)

def merged_scores(scores, intervals, merge_type):
    # A generator that returns merged intervals/scores
    # Scores should have shape: #examples x #categories x #interval_size
    # Second dimension can be omitted for a 1D signal
    signal_dims = scores.ndim - 1
    assert signal_dims in {1, 2}

    # Only support max for now
    assert merge_type == 'max'
    score_first_dim = 1 if signal_dims == 1 else scores.shape[1]

    dtype = scores.dtype

    sort_idx, sorted_intervals = \
        zip(*sorted(enumerate(intervals),
                    key=lambda item: interval_key(item[1])))
    sorted_intervals = BedTool(sorted_intervals)

    # Require at least 1bp overlap
    # Explicitly convert to list otherwise it will keep opening a file when
    # retrieving an index resulting in an error (too many open files)
    interval_clust = list(sorted_intervals.cluster(d=-1))
    for _, group in groupby(izip(sort_idx, interval_clust),
                            key=lambda item: item[1].fields[-1]):
        idx_interval_pairs = list(group)
        group_idx, group_intervals = zip(*idx_interval_pairs)

        if len(idx_interval_pairs) == 1:
            yield group_intervals[0], scores[group_idx[0], ...]
        else:
            group_chrom = group_intervals[0].chrom
            group_start = min(interval.start for interval in group_intervals)
            group_stop = max(interval.stop for interval in group_intervals)

            # This part needs to change to support more merge_types (e.g. mean)
            group_score = np.full((score_first_dim, group_stop - group_start),
                                  -np.inf, dtype)
            for idx, interval in idx_interval_pairs:
                slice_start = interval.start - group_start
                slice_stop = slice_start + (interval.stop - interval.start)
                group_score[..., slice_start:slice_stop] = \
                    np.maximum(group_score[..., slice_start:slice_stop],
                               scores[idx, ...])
            if signal_dims == 1:
                group_score = group_score.squeeze(axis=0)
            yield Interval(group_chrom, group_start, group_stop), group_score
            
def interval_score_pairs(intervals, scores, merge_type):
    return (izip(intervals, scores) if merge_type is None
            else merged_scores(scores, intervals, merge_type))

def _write_1D_deeplift_track(scores, intervals, file_prefix, merge_type='max',
                             CHROM_SIZES='/mnt/data/annotations/by_release/hg19.GRCh37/hg19.chrom.sizes'):
    assert scores.ndim == 2

    bedgraph = file_prefix + '.bedGraph'
    bigwig = file_prefix + '.bw'

    print 'Writing 1D track of shape: {}'.format(scores.shape)
    print 'Writing to file: {}'.format(bigwig)

    with open(bedgraph, 'w') as fp:
        for interval, score in interval_score_pairs(intervals, scores,
                                                    merge_type):
            chrom = interval.chrom
            start = interval.start
            for score_idx, val in enumerate(score):
                fp.write('%s\t%d\t%d\t%g\n' % (chrom,
                                               start + score_idx,
                                               start + score_idx + 1,
                                               val))
    print 'Wrote bedgraph.'

    try:
        output = subprocess.check_output(
            ['wigToBigWig', bedgraph, CHROM_SIZES, bigwig],
            stderr=subprocess.STDOUT)
        print 'wigToBigWig output: {}'.format(output)
    except subprocess.CalledProcessError as e:
        print 'wigToBigWig terminated with exit code {}'.format(
            e.returncode)
        print 'output was:\n' + e.output

    print 'Wrote bigwig.'

In [20]:
model_dir = os.path.join("models", "wgan")
log_dir = os.path.join("logs", "wgan")
srv_dir = os.path.join("/srv", "www", "kundaje", "jesikmin", "wgan")
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
if not os.path.exists(srv_dir):
    os.makedirs(srv_dir)

In [21]:
from keras.layers.merge import _Merge
from keras.optimizers import RMSprop
class RandomWeightedAverage(_Merge):
    """Provides a (random) weighted average between real and generated image samples"""
    def _merge_function(self, inputs):
        weights = K.random_uniform((batch_size, 1, 1))
        return (weights * inputs[0]) + ((1 - weights) * inputs[1])

In [34]:
# Our Improved WGAN Model
from functools import partial
class WGAN():
    def __init__(self, window_size, sample_num):
        # Basic parameters
        # 1) Window size of input/ouput signal
        self.window_size = window_size
        # 2) Number of channels
        self.channels = 5
        # 3) Input and Output shape
        self.input_shape = (self.window_size, self.channels,)
        self.output_shape = (self.window_size, 1,)
        # 4) Total number of samples
        self.sample_num = sample_num
        
        # Following parameter and optimizer set as recommended in paper
        self.n_critic = 5
        optimizer = RMSprop(lr=0.00005)

        # Build the generator and discriminator
        self.generator = self.build_generator()
        self.discriminator = self.build_discriminator()

        #-------------------------------
        # Construct Computational Graph
        #       for Discriminator
        #-------------------------------

        # Freeze generator's layers while training discriminator
        self.generator.trainable = False

        # Real histone marks (real sample)
        real_input = Input(shape=self.output_shape)

        # Noise input
        # Note: In our case, noise input is in same shape of the real input
        z_disc = Input(shape=self.input_shape)
        # Generate image based of noise (fake sample)
        fake_input = self.generator(z_disc)

        # Discriminator determines validity of the real and fake images
        fake = self.discriminator(fake_input)
        real = self.discriminator(real_input)

        merged_input = RandomWeightedAverage()([real_input, fake_input])
        # Determine validity of weighted sample
        valid_merged = self.discriminator(merged_input)

        # Use Python partial to provide loss function with additional
        # 'averaged_samples' argument
        partial_gp_loss = partial(self.gradient_penalty_loss,
                          averaged_samples=merged_input)
        partial_gp_loss.__name__ = 'gradient_penalty' # Keras requires function names

        self.discriminator_model = Model(inputs=[real_input, fake_input],
                            outputs=[real, fake, valid_merged])
        self.discriminator_model.compile(loss=[self.wasserstein_loss,
                                              self.wasserstein_loss,
                                              partial_gp_loss],
                                        optimizer=optimizer,
                                        loss_weights=[1, 1, 10])
        #-------------------------------
        # Construct Computational Graph
        #         for Generator
        #-------------------------------

        # For the generator we freeze the discriminator's layers
        self.discriminator.trainable = False
        self.generator.trainable = True

        # Sampled noise for input to generator
        z_gen = Input(shape=self.input_shape)
        # Generate images based of noise
        img = self.generator(z_gen)
        # Discriminator determines validity
        valid = self.discriminator(img)
        # Defines generator model
        self.generator_model = Model(z_gen, valid)
        self.generator_model.compile(loss=self.wasserstein_loss, optimizer=optimizer)

    def gradient_penalty_loss(self, y_true, y_pred, averaged_samples):
        """
        Computes gradient penalty based on prediction and weighted real / fake samples
        """
        gradients = K.gradients(y_pred, averaged_samples)[0]
        # compute the euclidean norm by squaring ...
        gradients_sqr = K.square(gradients)
        #   ... summing over the rows ...
        gradients_sqr_sum = K.sum(gradients_sqr,
                                  axis=np.arange(1, len(gradients_sqr.shape)))
        #   ... and sqrt
        gradient_l2_norm = K.sqrt(gradients_sqr_sum)
        # compute lambda * (1 - ||grad||)^2 still for each single sample
        gradient_penalty = K.square(1 - gradient_l2_norm)
        # return the mean as loss over all the batch samples
        return K.mean(gradient_penalty)

    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)

    def build_generator(self):

        noise_shape = self.input_shape

        model = Sequential()

        model.add(Conv1D(hidden_filters_1,
                         hidden_kernel_size_1,
                         padding="same",
                         strides=1,
                         input_shape=noise_shape,
                         activation='relu',
                         name='gen_conv1d_1'))
        model.add(BatchNormalization(momentum=0.8))

        # 2) 1 * 16 Conv1D layers with Linear
        # NOTE: All same padding
        model.add(Conv1D(output_filters,
                         output_kernel_size,
                         padding='same',
                         strides=1,
                         activation='linear',
                         name='gen_conv1d_output'))
    
        print "GENERATOR"
        model.summary()

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):
        model = Sequential()
        
        model.add(Conv1D(hidden_filters_1,
                         200,
                         padding="valid",
                         strides=1,
                         input_shape=self.output_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(dropout_rate))
        model.add(BatchNormalization(momentum=0.8))
        
        # 2) Average Pooling, Flatten, Dense, and LeakyRelu
        model.add(AveragePooling1D(25))
        model.add(Flatten())
        model.add(Dense(int(window_size/16)))
        model.add(LeakyReLU(alpha=0.2)) 
        
        # 3) Final output with sigmoid
        model.add(Dense(1, activation='sigmoid'))
        
        print "Discriminator"
        model.summary()

        img = Input(shape=self.output_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size, X_train, y_train):
        
        max_pearson = -1.0
        
        # Adversarial ground truths
        valid = -np.ones((batch_size, 1))
        fake = np.ones((batch_size, 1))
        dummy = np.zeros((batch_size, 1)) # Dummy gt for gradient penalty
        for epoch in range(epochs):
            d_losses, d_accuracies, g_losses = [], [], []
            
            for _minibatch_idx in range(128):
                for _ in range(self.n_critic):

                    # ---------------------
                    #  Train Discriminator
                    # ---------------------

                    # Select a random batch of images
                    idx = np.random.randint(0, y_train.shape[0], batch_size)
                    imgs = y_train[idx]

                    # Sample generator input
                    noise = X_train[idx]
                    gen_imgs = self.generator.predict(noise)
                    
                    print imgs.shape
                    print gen_imgs.shape
                    print valid.shape

                    # Train the discriminator
                    d_loss = self.discriminator_model.train_on_batch([imgs, gen_imgs],
                                                                    [valid, fake, dummy])
                    # ---------------------
                    # Store loss and accuracy
                    # ---------------------
                    d_losses.append(1-d_loss[0])
                    d_accuracies.append(d_loss[1])
                    
                # ---------------------
                #  Train Generator
                # ---------------------

                # Sample generator input
                idx = np.random.randint(0, X_train.shape[0], batch_size)
                noise = X_train[idx]
                # Train the generator
                g_loss = self.generator_model.train_on_batch(noise, valid)
                g_losses.append(1-g_loss[0])
            
            # ---------------------
            # Convert each histories into numpy arrays to get means
            # ---------------------
            d_losses = np.array(d_losses)
            d_accuracies = np.array(d_accuracies)
            g_losses = np.array(g_losses)
            
            # ---------------------
            # Get generator's prediction and compute overall pearson on train set
            # ---------------------
            predictions = self.generator.predict(X_train).flatten()
            avg_pearson = pearsonr(predictions, y_train.flatten())
            print "Pearson R on Train set: {}".format(avg_pearson)
            
            # ---------------------
            # Get generator's prediction and compute overall pearson on validation set
            # ---------------------
            val_predictions = self.generator.predict(X_val).flatten()
            avg_val_pearson = pearsonr(val_predictions, y_val.flatten())
            print "Pearson R on Val set: {}".format(avg_val_pearson)
            
            # if current pearson on validation set is greatest so far, update the max pearson,
            # and 
            if max_pearson < avg_val_pearson:
                print "Perason on val improved from {} to {}".format(max_pearson, avg_val_pearson)
                _write_1D_deeplift_track(predictions.reshape(int(sample_num*0.8), self.window_size),
                                         normalized_day0_intervals[:int(sample_num*0.8)], os.path.join(srv_dir, 'train'))
                _write_1D_deeplift_track(val_predictions.reshape(int(sample_num*0.2), self.window_size),
                                         normalized_day0_intervals[int(sample_num*0.8):sample_num], os.path.join(srv_dir, 'val'))
                f = open(os.path.join(srv_dir, 'meta.txt'), 'wb')
                f.write(str(epoch) + " " + str(avg_pearson) + "  " + str(avg_val_pearson))
                f.close()
                max_pearson = avg_val_pearson
            
            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_losses.mean(), 100.0*d_accuracies.mean(), g_losses.mean()))

In [35]:
# helper function for computing Pearson R in Keras
def pearson(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x)
    my = K.mean(y)
    xm, ym = x-mx, y-my
    r_num = K.sum(np.multiply(xm,ym))
    r_den = K.sqrt(np.multiply(K.sum(K.square(xm)), K.sum(K.square(ym))))
    r = r_num / r_den
    r = K.maximum(K.minimum(r, 1.0), -1.0)
    return K.square(r)

In [36]:
print "Fitting the model..."
train_index = int(sample_num*0.8)
val_index = sample_num
test_index = int(sample_num*1.1) 
X_train, y_train = atac_seq_day0[:train_index], histone_mark_day0[:train_index]
X_val, y_val = atac_seq_day0[train_index:val_index], histone_mark_day0[train_index:val_index]
X_test, y_test = atac_seq_day0[val_index:test_index], histone_mark_day0[val_index:test_index]

gan = WGAN(window_size, sample_num)
gan.train(num_epochs, batch_size, X_train, y_train)

Fitting the model...
GENERATOR
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gen_conv1d_1 (Conv1D)        (None, 2001, 32)          320192    
_________________________________________________________________
batch_normalization_9 (Batch (None, 2001, 32)          128       
_________________________________________________________________
gen_conv1d_output (Conv1D)   (None, 2001, 1)           1025      
Total params: 321,345
Trainable params: 321,281
Non-trainable params: 64
_________________________________________________________________
Discriminator
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, 1802, 32)          6432      
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 1802, 32)          0         
________________________

/users/jesikmin/anaconda2/envs/genomelake_env/lib/python2.7/site-packages/keras/engine/topology.py:1590: UserWarning: Model inputs must come from a Keras Input layer, they cannot be the output of a previous non-Input layer. Here, a tensor specified as input to "model_15" was not an Input tensor, it was generated by layer model_13.
Note that input tensors are instantiated via `tensor = Input(shape)`.
The tensor that caused the issue was: model_13/sequential_9/gen_conv1d_output/add:0
  str(x.name))


AssertionError: 